###Downloading libraries

In [0]:
!pip install kaggle

     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 77 kB 4.0 MB/s 
     |████████████████████████████████| 78 kB 6.5 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77732 sha256=53b8f1e319da33096c62b3438081ea233ffb34fea26eecbfbe1136ad12753182
  Stored in directory: /root/.cache/pip/wheels/9c/45/15/6d6d116cd2539fb8f450d64b0aee4a480e5366bb11b42ac763
Successfully built kaggle
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-ce213351-6f0e-4e42-b468-cbf8b9afd234/bin/python -m pip install --upgrade pip' command.


###Downloading the NYT dataset

In [0]:
import os

dbutils.fs.cp('/FileStore/kaggle.json', 'file:/kaggle/kaggle.json')
os.environ['KAGGLE_CONFIG_DIR'] = '/kaggle'

!kaggle datasets download -d aryansingh0909/nyt-articles-21m-2000-present --force
!unzip nyt-articles-21m-2000-present.zip

dbutils.fs.mv("file:/databricks/driver/nyt-metadata.csv", "/FileStore/nyt-metadata.csv")

100%|███████████████████████████████████████▊| 867M/871M [00:17<00:00, 47.5MB/s]
100%|████████████████████████████████████████| 871M/871M [00:24<00:00, 36.8MB/s]
Archive:  nyt-articles-21m-2000-present.zip
  inflating: nyt-metadata.csv        


###Reading .csv file to DataFrame

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, DateType, FloatType, ArrayType, MapType

nyt_schema = StructType([
    StructField('abstract', StringType(), True), 
    StructField('web_url', StringType(), False), 
    StructField('snippet', StringType(), True), 
    StructField('lead_paragraph', StringType(), True), 
    StructField('print_section', StringType(), True), 
    StructField('print_page', FloatType(), False), 
    StructField('source', StringType(), False), 
    #StructField('multimedia', ArrayType(MapType(StringType(), StringType())), True), 
    StructField('multimedia', StringType(), True), 
    #StructField('headline', MapType(StringType(), StringType()), True), 
    StructField('headline', StringType()), 
    #StructField('keywords',  ArrayType(MapType(StringType(), StringType())), True), 
    StructField('keywords', StringType(), True),
    StructField('pub_date', DateType(), False), 
    StructField('document_type', StringType(), True), 
    StructField('news_desk', StringType(), True), 
    StructField('section_name', StringType(), True), 
    #StructField('byline', MapType(StringType(), StringType()), True), 
    StructField('byline', StringType(), True),
    StructField('type_of_material', StringType(), True), 
    StructField('_id', StringType(), True), 
    StructField('word_count', FloatType(), True), 
    StructField('uri', StringType(), False), 
    StructField('subsection_name', StringType(), True),
    StructField('corrupted', StringType(), True)
])

nyt_articles = (spark
    .read
    .option('header', 'true')
    .option('dateFormat', 'yyyy-mm-dd')
    .schema(nyt_schema)
    .csv('/FileStore/nyt-metadata.csv'))


###Data processing

In [0]:
from pyspark.sql.functions import from_json, schema_of_json
nyt_articles.shuffle().take(3)
#(nyt_articles.withColumn("multimedia", to_json("multimedia"))
#             .withColumn("headline", to_json("headline"))
#             .withColumn("keywords", to_json("keywords"))
#             .withColumn("byline", to_json("byline"))
#             .groupBy('corrupted')
#             .count()
#             .show())

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3411769209486607>:2
      1 from pyspark.sql.functions import from_json, schema_of_json
----> 2 nyt_articles.shuffle().take(3)

NameError: name 'nyt_articles' is not defined

###Checking for errors

In [0]:
nyt_articles.groupBy('corrupted').count().show()

+--------------------+-----+
|           corrupted|count|
+--------------------+-----+
|      Editorial Desk| 1090|
|{'original': 'By ...|   38|
|{'original': 'By ...|    1|
|{'original': 'By ...|    3|
|{'original': 'By ...|    1|
|{'original': 'By ...|   56|
|{'original': 'By ...|    8|
|{'original': 'By ...|   68|
|{'original': 'By ...|   14|
|{'original': 'By ...|    7|
|nyt://article/3bd...|    1|
|nyt://article/82a...|    1|
|{'original': 'By ...|    3|
|{'original': 'By ...|    1|
|             HAROLD'|    1|
|{'original': 'By ...|    1|
|{'original': 'By ...|    1|
|2000-10-27 05:00:...|    1|
|nyt://article/64a...|    1|
|2000-11-12 05:00:...|    3|
+--------------------+-----+
only showing top 20 rows

